This notebook tests the package `mlxtend`

In notebook 1.4, there is a simple stacked model that used two logistic regressions. Goal for this notebook is to reproduce the results via mlxtend API, and also add cross-validation to it.

In [1]:
import helpers
import pandas as pd
import pickle
import numpy as np
import math
import librosa.feature as lf
from sklearn import model_selection
from sklearn import linear_model
from mlxtend.classifier import StackingClassifier

## Stacking

the two simple models and the meta model in this case are all logistic regressions.

### Start from Make Two Models in notebook 1.4

In [2]:
TRAIN_CSV = 'data/external/train.csv'
TRAIN_FILES = 'data/external/audio_train'

tags = list(pd.read_csv(TRAIN_CSV).label.unique())

label_reference = pd.Series(tags).rename('label').to_frame().reset_index().rename({'index': 'label_index'}, axis=1)

In [3]:
file_labels = (
    pd.read_csv(TRAIN_CSV)
    .drop(['manually_verified'], axis=1)
    .rename({'fname': 'name'}, axis=1)
    .merge(label_reference, on=['label'])
)

Make a dataset that has both chroma_stft features and mfcc features

In [4]:
train_chroma_stft = pd.concat([pd.read_pickle('data/interim/{}-1-sec/{}.pkl'.format('chroma_stft', name)) for name in tags])
train_mfcc = pd.concat([pd.read_pickle('data/interim/{}-1-sec/{}.pkl'.format('mfcc', name)) for name in tags])

name_features = train_chroma_stft.merge(train_mfcc, on=['name'], how='inner')

In [5]:
name_features.shape

(9473, 1409)

In [6]:
df_train = (
    file_labels
    .merge(name_features, on=['name'], how='inner')
)

In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    df_train.drop(['name', 'label', 'label_index'], axis=1).values,
    df_train['label_index'].values,
    test_size=0.25, 
    random_state=707,
    stratify=df_train['label_index'].values
)

Find the index for chroma_stft and mfcc columns

In [8]:
df_train_feature_only = df_train.drop(['name', 'label', 'label_index'], axis=1)

In [9]:
df_train_feature_only.columns

Index(['chroma_stft_0', 'chroma_stft_1', 'chroma_stft_2', 'chroma_stft_3',
       'chroma_stft_4', 'chroma_stft_5', 'chroma_stft_6', 'chroma_stft_7',
       'chroma_stft_8', 'chroma_stft_9',
       ...
       'mfcc_870', 'mfcc_871', 'mfcc_872', 'mfcc_873', 'mfcc_874', 'mfcc_875',
       'mfcc_876', 'mfcc_877', 'mfcc_878', 'mfcc_879'],
      dtype='object', length=1408)

In [10]:
chromas_ind = tuple(i for i, colname in enumerate(df_train_feature_only.columns) if 'chroma_stft' in colname)
mfcc_ind = tuple(i for i, colname in enumerate(df_train_feature_only.columns) if 'mfcc' in colname)

Feed the features into models

In [11]:
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector

In [12]:
pipe_chromas = make_pipeline(ColumnSelector(cols=chromas_ind),linear_model.LogisticRegression(random_state=123))
pipe_mfcc = make_pipeline(ColumnSelector(cols=mfcc_ind),linear_model.LogisticRegression(tol=0.001, random_state=123))

In [13]:
sclf = StackingClassifier(classifiers=[pipe_chromas, pipe_mfcc], 
                          meta_classifier=linear_model.LogisticRegression(random_state=123))

In [14]:
sclf.fit(X_train, y_train)

StackingClassifier(average_probas=False,
          classifiers=[Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,...ty='l2', random_state=123, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False))])],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [15]:
sclf

StackingClassifier(average_probas=False,
          classifiers=[Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(cols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,...ty='l2', random_state=123, solver='liblinear', tol=0.001,
          verbose=0, warm_start=False))])],
          meta_classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          store_train_meta_features=False, use_clones=True,
          use_features_in_secondary=False, use_probas=False, verbose=0)

In [17]:
sclf.score(X_test, y_test)

0.10257492612916842

This score is way lower than handcrafted stack (30%) in notebook 1.4 --- needs debugging